<a href="https://colab.research.google.com/github/Vish4github/Projects/blob/master/News_Category_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install fastai==0.7.0

In [0]:
!pip3 install torchvision
!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [0]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [0]:
from torch import * 
!pip install Pillow==4.1.1
!pip install image
%matplotlib inline
from fastai.imports import *

In [0]:
!pip install "torchtext==0.2.3"

In [0]:
!pip3 install spacy 
!python -m spacy download en

In [0]:
!nvcc --version

In [0]:
import torch
torch.cuda.is_available()
torch.backends.cudnn.enabled
#!nvidia-smi
#!python --version

In [0]:
!pip3 install kaggle

In [0]:
#!pip install fastai
!pip install fastai --upgrade


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [0]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/Data/News_Category_Dataset_v2.json"
df =  pd.read_json(path,lines=True)
df.head()

In [0]:
cates = df.groupby('category')
print("total categories:", cates.ngroups)
print(cates.size())

In [0]:
df.category = df.category.map(lambda x: "WORLDPOST" if x == "THE WORLDPOST" else x)

In [0]:
df['text'] = df.headline + " " + df.short_description

In [0]:
df_text = df[['text','category']]
df_text.head()

In [0]:
from sklearn.model_selection import train_test_split
train_df,valid_df=train_test_split(df_text, shuffle=True, test_size=0.25)

len(train_df),len(valid_df)

In [0]:
train_df.head(),valid_df.head()

In [0]:
# Language model data
data_lm = TextLMDataBunch.from_df(path,train_df=train_df,valid_df=valid_df,text_cols='text', label_cols='category',bs=16)

# Classification model data
data_clas = TextClasDataBunch.from_df(path, train_df=train_df,valid_df=valid_df, vocab=data_lm.train_ds.vocab, text_cols = 'text', label_cols = 'category',bs=16)


In [0]:
data_clas.show_batch()

In [0]:
#
data_lm.train_ds

In [0]:
data_lm.vocab.itos[448]

In [0]:
data_lm.train_ds[0][0].data[:10]

In [0]:
len(data_lm.vocab.itos),len(data_lm.train_ds)

In [0]:
data_lm.save('/content/drive/My Drive/Data/data_lm_export.pkl')
data_clas.save('/content/drive/My Drive/Data/data_clas_export.pkl')

In [0]:
bs = 48
path = '/content/drive/My Drive/Data/'

In [0]:
data_lm = load_data(path, 'data_lm_export.pkl')
data_clas = load_data(path, 'data_clas_export.pkl', bs=16)

In [0]:
data_clas.show_batch()

In [0]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)
#learn.fit_one_cycle(1, 1e-2)

In [0]:
vocab = data_lm.vocab

In [0]:
vocab.stoi["trump"]

In [0]:
awd = learn_lm.model[0]

In [0]:
enc = learn_lm.model[0].encoder

In [0]:
enc.weight.size()

In [0]:
TEXT = "The color of the sky is"
N_WORDS = 30
N_SENTENCES = 1

In [0]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [0]:
learn_lm.lr_find()

In [0]:
learn_lm.recorder.plot(skip_end=15)

In [0]:
lr = 1e-2
lr *= bs/48

In [0]:
learn_lm.to_fp16()

In [0]:
learn_lm.fit_one_cycle(1, lr*10, moms=(0.8,0.7))

In [0]:
learn_lm.save('fit_1')

In [0]:
learn_lm.load('fit_1')

In [0]:
learn_lm.unfreeze()

In [0]:
learn_lm.fit_one_cycle(10, lr, moms=(0.8,0.7))

In [0]:
learn_lm.predict("This is a review about", n_words=20)

In [0]:
learn_lm.save_encoder('ft_enc')

Classifier

In [0]:
data_clas.show_batch()

In [0]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

In [0]:
data_clas.show_batch()

In [0]:
learn.fit_one_cycle(1, 1e-2)

In [0]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

In [0]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

In [0]:
learn.predict("This was a great movie!")